In [10]:
import train as t
import tensorflow as tf

import importlib
#importlib.reload(t)

from enum import Enum

from functools import reduce

In [11]:
# Create the vocabulary
token_vocabulary = ["**end**", "**start**", "**unknown**"]

token_vocabulary.extend(t.get_vocabulary("train"))

target_tokens = token_vocabulary  # TODO: Refactor this. Currently duplicate naming

token_vocab_size = len(target_tokens)
# todo: document what was lifted from tutorials and what we wrote ourselves
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_target_token_index = dict(
    (i, char) for char, i in target_token_index.items())



print("\n ======================= Loading Data =======================")
#new cell

train_dataset = t.get_data_somehow('train', True, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_train_num_samples'] , target_token_index)
val_dataset = t.get_data_somehow('val', True, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_val_num_samples'], target_token_index)

train_encoder_input_data_batches = train_dataset[0]
train_target_texts_batches = train_dataset[1]
train_sequence_lengths_batches = train_dataset[2]
train_decoder_input_data_batches = train_dataset[3]
train_decoder_target_data_batches = train_dataset[4]

val_encoder_input_data_batches = val_dataset[0]
val_target_texts_batches = val_dataset[1]
val_sequence_lengths_batches = val_dataset[2]
val_decoder_input_data_batches = val_dataset[3]
val_decoder_target_data_batches = val_dataset[4]
print("\n ======================= Data Loaded =======================")


 ======================= Loading Data =======================
dumped
dumped

 ======================= Data Loaded =======================


In [12]:
tf.reset_default_graph()


embedding_decoder, decoder_cell, decoder_initial_state, projection_layer, img \
= t.create_graph(token_vocab_size, t.hparams['num_units'], t.hparams['use_attention'], t.hparams['use_encoding_average_as_initial_state'], False)


INFO:tensorflow:Summary name conv2d/kernel:0/gradient is illegal; using conv2d/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d/bias:0/gradient is illegal; using conv2d/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_1/kernel:0/gradient is illegal; using conv2d_1/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d_1/bias:0/gradient is illegal; using conv2d_1/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0/gradient is illegal; using conv2d_2/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d_2/bias:0/gradient is illegal; using conv2d_2/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_3/kernel:0/gradient is illegal; using conv2d_3/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2d_3/bias:0/gradient is illegal; using conv2d_3/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2d_4/kernel:0/gradient is illegal; using conv2d_4/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2

In [13]:
sess = tf.Session()
t.initialize_variables(sess, restore=True, path="../output/checkpoints" + '/model_127.ckpt')



reinitializing


In [15]:
batch = train_encoder_input_data_batches[0]

little_variable, logits = t.predict_batch(sess,
                  batch,
                  target_token_index,
                  embedding_decoder,
                  decoder_cell,
                  decoder_initial_state,
                  projection_layer,
                  img)

little_variable

array([[364,   5,   5,   5,   5,   5, 382, 382, 382, 382, 382, 382, 382,
        382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382,
        382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382,
        382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382],
       [364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364,
        364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364,
        364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364,
        364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364],
       [364, 364, 364,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
          5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
          5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
          5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5],
       [364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364,
        364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364, 364,
     

In [16]:
def get_token_seq(int_sequence):
    
    output_string = ""
    for value in int_sequence:
        output_string += reverse_target_token_index[value] + " "
        if reverse_target_token_index[value] == "**end**":
            break
    return output_string

for idx, seq in enumerate(train_target_texts_batches[0]):
    
    print("Output: ")
    print(get_token_seq(little_variable[idx]))
    print("")
    print("Ground truth: ")
    print(train_target_texts_batches[0][idx])
    print("")

Output: 
^ & & & & & n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n 

Ground truth: 
**start** \, ^ { * } d \, ^ { * } H = \kappa \, ^ { * } d \phi = J _ { B } .

Output: 
^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 

Ground truth: 
**start** \partial _ { \mu } ( F ^ { \mu \nu } - e j ^ { \mu } x ^ { \nu } ) = 0 .

Output: 
^ ^ ^ & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & & 

Ground truth: 
**start** L _ { 0 } = \Phi ( w ) = \bigtriangleup \Phi ( w ) ,

Output: 
^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 

Ground truth: 
**start** ( { \cal L } _ { a } g ) _ { i j } = 0 , \ \ \ \ ( { \cal L } _ { a } H ) _ { i j k } = 0 ,

Output: 
^ & & ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 

Ground truth: 
**start** S _ { s t a t } = 2 \pi \sqrt {

In [8]:
class DS(Enum):
    ENCODER_INPUTS = 0
    TARGET_TEXTS = 1
    LENGTHS = 2
    DECODER_INPUTS = 3
    DECODER_TARGETS = 4
    

In [ ]:
sess = tf.Session()


In [ ]:
saver.restore(sess=sess, save_path="../output/checkpoints/inner.ckpt")

In [ ]:
logits = ????????

In [ ]:
#not done
def predict(set_, loss_tensor, logits_tensor, sess, labels=None):
    n_batches = len(set_[DS.LENGTHS])
    
    #returns the logits for a batch
    def predict_batch_tensor(set_, idx):
        input_ = {img: set_[Batch.ENCODER_INPUTS][idx],
                  decoder_lengths: set_[DS.TARGET_TEXT],
                  decoder_inputs: set_[DS.DECODER_INPUT],
                  decoder_outputs: set_[DS.DECODER_TARGET]
                 }
        
    
    
    
    

In [ ]:
correct = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(labels, axis=1))
        
    
    total = decoder_lengths.shape[0]

    loss, correct, total = sess.run([loss_tensor, accuracy_tensor, total], feed_dict=input_)
    return loss, correct, total

    def 

In [ ]:
def get_validation_loss():
    #num_val_batches = len(val_sequence_lengths_batches)
    val_loss = 0    
    for i in range(num_val_batches):
        input_data = {img: val_encoder_input_data_batches[i],
                                        decoder_lengths: val_sequence_lengths_batches[i],
                                         decoder_inputs: val_decoder_input_data_batches[i],
                                          decoder_outputs: val_decoder_target_data_batches[i],
                                         }
   
        output_tensors = [train_loss]
        loss = sess.run(output_tensors, 
                               feed_dict=input_data)
        
        print(loss)
        val_loss = val_loss + loss[0]
        
    val_loss = val_loss / len(val_decoder_input_data_batches[i])
    return val_loss